In [2]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "1677436144475019467", 2, "feature")

## Vecchia funzione

In [3]:
function embedStruct(n::Int)
	function embedStruct0(self::Lar.Struct,suffix::String="New")
		if n==0
			return self, length(self.box[1])
		end
		cloned = Lar.Struct()
		cloned.box = [ [self.body[i].box[1];zeros(dimadd)],
						[self.body[i].box[2];zeros(dimadd)] ]
		cloned.name = self.name*suffix
		cloned.category = self.category
		cloned.dim = self.dim+n
		cloned = Lar.embedTraversal(cloned,self,n,suffix)
		return cloned
	end
	return embedStruct0
end

embedStruct (generic function with 1 method)

## Nuove funzioni

In [20]:
function embedStruc(n)
	function embedStruct0(self)
        suffix = "-New"
		if n==0
			return self, length(self.box[1])
		end
		cloned = Lar.Struct()
		cloned.box = hcat((self.box,[fill([0],n),fill([0],n)]))
		cloned.name = self.name*suffix
		cloned.category = self.category
		cloned.dim = self.dim+n
		cloned = Lar.embedTraversal(cloned,self,n,suffix)
		return cloned
	end
	return embedStruct0
end

embedStruc (generic function with 1 method)

In [21]:
@btime embedStruc(1)(x)

  11.399 μs (33 allocations: 2.45 KiB)


LinearAlgebraicRepresentation.Struct(Any[[([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0; 0.0 0.0 0.0 0.0], [[1, 2, 3, 4]])]], [([[0.0; 0.0], [1.0; 1.0]], [[[0]], [[0]]])], "1677436144475019467-New", 3, "feature")

In [23]:
@benchmark embedStruc(1)(x)

BenchmarkTools.Trial: 
  memory estimate:  2.44 KiB
  allocs estimate:  33
  --------------
  minimum time:     11.499 μs (0.00% GC)
  median time:      19.701 μs (0.00% GC)
  mean time:        33.794 μs (0.00% GC)
  maximum time:     4.485 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [24]:
@code_warntype embedStruc(1)(x)

Variables
  #self#::var"#embedStruct0#16"{Int64}
  self::LinearAlgebraicRepresentation.Struct
  suffix::String
  cloned::Any

Body::Any
1 ─       Core.NewvarNode(:(cloned))
│         (suffix = "-New")
│   %3  = Core.getfield(#self#, :n)::Int64
│   %4  = (%3 == 0)::Bool
└──       goto #3 if not %4
2 ─ %6  = Base.getproperty(self, :box)::Any
│   %7  = Base.getindex(%6, 1)::Any
│   %8  = Main.length(%7)::Any
│   %9  = Core.tuple(self, %8)::Tuple{LinearAlgebraicRepresentation.Struct,Any}
└──       return %9
3 ─ %11 = Base.getproperty(Main.Lar, :Struct)::Any
│         (cloned = (%11)())
│   %13 = Base.getproperty(self, :box)::Any
│   %14 = Base.vect(0)::Array{Int64,1}
│   %15 = Core.getfield(#self#, :n)::Int64
│   %16 = Main.fill(%14, %15)::Array{Array{Int64,1},1}
│   %17 = Base.vect(0)::Array{Int64,1}
│   %18 = Core.getfield(#self#, :n)::Int64
│   %19 = Main.fill(%17, %18)::Array{Array{Int64,1},1}
│   %20 = Base.vect(%16, %19)::Array{Array{Array{Int64,1},1},1}
│   %21 = Core.tuple(%13, %20

## Modifiche